# Test Dataset (Work in Progress)

Placeholder for when all 20 nanopore samples have been sequenced and aligned and modcalled. We will put them here and merge with main dataset.

1. Converting unaligned CRAM to FASTQ for Minimap2 aligment:

    ```bash
    samtools bam2fq -@ 32 -T 1 --reference ./ref/hg38.fna cram/uf_hembank_${ID}.pass.cram | \
    minimap2 -y -t 32 -ax lr:hq ref/hg38.fna - > sam/uf_hembank_${ID}.sam
    ```

2. Converting SAM to BAM, sorting, and indexing:

    ```bash
    samtools view -@ 32 -bS sam/uf_hembank_${ID}.sam | \
    samtools sort -@ 32 - --write-index -o bam/uf_hembank_${ID}.bam -O BAM --reference ref/hg38.fna
    ```

3. Creating a BedMethyl file with Modkit
    
    ```bash
    modkit pileup bam/uf_hembank_${ID}.bam bed/uf_hembank_${ID}.bed -t 32 --combine-strands --cpg --ignore h --ref ref/hg38.fna
    ```

## Where data at?

In [1]:
import pandas as pd
import glob

# Get a list of all CSV files in the directory
file_list = glob.glob('../../pacmap/*.csv')

# Create an empty list to store the dataframes
dfs = []

# Read each CSV file and append it to the list
for file in file_list:
    df = pd.read_csv(file, index_col=0)
    dfs.append(df)

# Concatenate all dataframes in the list
df = pd.concat(dfs)

In [2]:
df

,cg00000109,cg00000236,cg00000292,cg00000363,cg00000622,cg00000658,cg00000714,cg00000721,cg00000734,cg00000769,...,cg21278787,cg22708233,cg23138682,cg23282051,cg24677744,cg25023752,cg26185531,cg26293201,cg26576875,cg07148304
uf_hembank_1852,1.000,0.913,0.615,0.174,0.0,0.964,0.250,1.000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
uf_hembank_1831,1.000,0.800,0.786,0.143,0.0,0.933,0.250,0.933,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
uf_hembank_1829,0.889,0.632,0.750,0.000,0.0,1.000,0.100,1.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,NaN
uf_hembank_1830,0.960,0.500,0.615,0.294,0.0,1.000,0.000,1.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
uf_hembank_1832,1.000,1.000,0.800,0.000,0.0,1.000,0.000,1.000,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
uf_hembank_1841,0.714,0.900,0.667,0.000,0.0,0.923,0.091,0.929,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.00,0.0,0.0,NaN


## Appendix 1. Retrospective lower coverage analysis

```bash
samtools view -@ 32 -bh -s ${subsampling_fraction} bam/uf_hembank_${ID}.bam > bam/subsampled_${ID}_01x.bam
```